#### Performing the Standard Verification Routine with the Caravagna-J-Theor-Biol-2010 publication.

In [1]:
import sys
import os

import numpy as np
from process_bigraph import Composite

from biosimulator_processes.io import read_report_outputs
from biosimulator_processes.helpers import plot_ode_output_data


sys.path.insert(0, '..')

In [2]:
omex_dir = '../examples/sbml-core/Caravagna-J-Theor-Biol-2010-tumor-suppressive-oscillations'
os.path.exists(omex_dir)
model_fp = os.path.join(omex_dir, 'Caravagna2010.xml')

In [3]:
# read "ground truth" output aka that which is PUBLISHED as the omex dir (including reports). Publication itself is a Validation Phase, according to the Standard Validation Routine

published_outputs = read_report_outputs(os.path.join(omex_dir, 'reports.h5'))

In [4]:
# published outputs are easily serializable

published_outputs.to_dict()

In [5]:
class Workflow(Composite):
    # TODO: finish this.
    def __init__(self, config=None, core=None):
        super().__init__(config, core)

In [6]:
from biosimulator_processes.steps.ode_simulation import AmiciStep

In [7]:
amici_step = AmiciStep(archive_filepath=omex_dir, sbml_filepath=model_fp, model_dir=os.getcwd())

In [8]:
amici_results = amici_step.update({})

In [9]:
amici_results

In [10]:
from biosimulator_processes.steps.ode_simulation import CopasiStep 

copasi_step = CopasiStep(archive_filepath=omex_dir)

In [11]:
copasi_results = copasi_step.update({})

In [12]:
copasi_results

In [13]:
plot_ode_output_data(amici_results, simulator_name='Amici')

In [14]:
plot_ode_output_data(copasi_results, simulator_name='COPASI')